In [1]:
import pandas as pd
import numpy as np

In [2]:
cg_data = pd.read_csv('campgrounds.csv')

In [3]:
cg_data.shape

(11, 8)

In [4]:
cg_data.head()

,facilityname,facilitylatitude,facilitylongitude,sites_available,firstcome,flush,shower,vault
0,Mazama,42.865508,-122.166149,\N,\N,1,1,\N
1,Claude Lewis Sno-Park-USFS,43.063217,-122.342876,\N,\N,\N,\N,1
2,Lost Creek,42.879145,-122.037881,\N,1,1,\N,\N
3,Hamaker,43.056641,-122.330322,\N,\N,\N,\N,1
4,Huckleberry Mountain,42.877807,-122.337174,\N,\N,\N,\N,1


In [5]:
for col in cg_data.columns:
    print(cg_data[col].unique())


['Mazama' 'Claude Lewis Sno-Park-USFS' 'Lost Creek' 'Hamaker'
 'Huckleberry Mountain' 'Natural Bridge -USFS' 'Farewell Bend'
 'Scott Creek' 'Thousand Springs Sno-Park-USFS' 'Union Creek'
 'Farewell Bend Campground']
[ 42.865508    43.063217    42.879145    43.056641    42.877807    42.889648
  42.919678    42.884588    42.913045    42.91027778  42.91623889]
[-122.166149   -122.342876   -122.037881   -122.330322   -122.337174
 -122.463867   -122.433105   -121.924772   -122.324644   -122.45
 -122.43533056]
['\\N' '0']
['\\N' '1' '0']
['1' '\\N']
['1' '\\N']
['\\N' '1' '0']


In [6]:
cg_data.sites_available.isnull()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
Name: sites_available, dtype: bool

In [7]:
import numpy as np

In [8]:
cg_data_clean = cg_data.replace('\\N', '')

In [9]:
cg_data_clean = cg_data_clean.drop(['sites_available','firstcome'], axis=1)

In [10]:
cg_data_clean.head()

,facilityname,facilitylatitude,facilitylongitude,flush,shower,vault
0,Mazama,42.865508,-122.166149,1,1,
1,Claude Lewis Sno-Park-USFS,43.063217,-122.342876,,,1
2,Lost Creek,42.879145,-122.037881,1,,
3,Hamaker,43.056641,-122.330322,,,1
4,Huckleberry Mountain,42.877807,-122.337174,,,1


In [11]:
cg_data_clean = cg_data_clean.replace({'flush': {'1':'Flush toilet', '0':''}})

In [12]:
cg_data_clean = cg_data_clean.replace({'shower': {'1':'Shower', '0':''}})
cg_data_clean = cg_data_clean.replace({'vault': {'1':'Vault toilet', '0':''}})

In [29]:
cg_data_clean = cg_data_clean.rename(columns={'facilitylatitude':'latitude', 'facilitylongitude':'longitude'})

In [30]:
cg_data_clean

,title,latitude,longitude,flush,shower,vault,description
0,Mazama,42.865508,-122.166149,Flush toilet,Shower,,"Flush toilet,Shower,"
1,Claude Lewis Sno-Park-USFS,43.063217,-122.342876,,,Vault toilet,",,Vault toilet"
2,Lost Creek,42.879145,-122.037881,Flush toilet,,,"Flush toilet,,"
3,Hamaker,43.056641,-122.330322,,,Vault toilet,",,Vault toilet"
4,Huckleberry Mountain,42.877807,-122.337174,,,Vault toilet,",,Vault toilet"
5,Natural Bridge -USFS,42.889648,-122.463867,,,Vault toilet,",,Vault toilet"
6,Farewell Bend,42.919678,-122.433105,,,,",,"
7,Scott Creek,42.884588,-121.924772,,,Vault toilet,",,Vault toilet"
8,Thousand Springs Sno-Park-USFS,42.913045,-122.324644,,,Vault toilet,",,Vault toilet"
9,Union Creek,42.910278,-122.450000,,,Vault toilet,",,Vault toilet"


Create a description field that combines our amenities into a single string

In [23]:
cg_data_clean['description'] = cg_data_clean[['flush','shower','vault']].apply(lambda x: ','.join(x), axis=1)
cg_data_clean

,title,facilitylatitude,facilitylongitude,flush,shower,vault,description
0,Mazama,42.865508,-122.166149,Flush toilet,Shower,,"Flush toilet,Shower,"
1,Claude Lewis Sno-Park-USFS,43.063217,-122.342876,,,Vault toilet,",,Vault toilet"
2,Lost Creek,42.879145,-122.037881,Flush toilet,,,"Flush toilet,,"
3,Hamaker,43.056641,-122.330322,,,Vault toilet,",,Vault toilet"
4,Huckleberry Mountain,42.877807,-122.337174,,,Vault toilet,",,Vault toilet"
5,Natural Bridge -USFS,42.889648,-122.463867,,,Vault toilet,",,Vault toilet"
6,Farewell Bend,42.919678,-122.433105,,,,",,"
7,Scott Creek,42.884588,-121.924772,,,Vault toilet,",,Vault toilet"
8,Thousand Springs Sno-Park-USFS,42.913045,-122.324644,,,Vault toilet,",,Vault toilet"
9,Union Creek,42.910278,-122.450000,,,Vault toilet,",,Vault toilet"


using assign seems to create a series object for description, as opposed to just setting the variable = the lambda in 147 above

In [ ]:
#cg_data_clean.assign(description = lambda row: row['flush'] + ", " + row['shower'] + ", " + row['vault'])

Use the facilityname as the title

Clean the extraneous commas from the description

In [16]:
def clean_description(description):
    description = description.strip()
    while((description.startswith(',') or description.endswith(',')) and len(description) > -1):
        if description.endswith(',') :
            description = description[0:len(description)-1]
        if description.startswith(',') :
            description = description[1:len(description)]   
        description = description.strip()
    print (description)
    return description
        

In [33]:
cg_data_clean['description'] = cg_data_clean.description.apply(lambda x: clean_description(x))

Flush toilet,Shower
Vault toilet
Flush toilet
Vault toilet
Vault toilet
Vault toilet

Vault toilet
Vault toilet
Vault toilet



In [34]:
cg_data_clean = cg_data_clean.rename(columns={'facilityname':'title'})

In [35]:
cg_data_clean

,title,latitude,longitude,flush,shower,vault,description
0,Mazama,42.865508,-122.166149,Flush toilet,Shower,,"Flush toilet,Shower"
1,Claude Lewis Sno-Park-USFS,43.063217,-122.342876,,,Vault toilet,Vault toilet
2,Lost Creek,42.879145,-122.037881,Flush toilet,,,Flush toilet
3,Hamaker,43.056641,-122.330322,,,Vault toilet,Vault toilet
4,Huckleberry Mountain,42.877807,-122.337174,,,Vault toilet,Vault toilet
5,Natural Bridge -USFS,42.889648,-122.463867,,,Vault toilet,Vault toilet
6,Farewell Bend,42.919678,-122.433105,,,,
7,Scott Creek,42.884588,-121.924772,,,Vault toilet,Vault toilet
8,Thousand Springs Sno-Park-USFS,42.913045,-122.324644,,,Vault toilet,Vault toilet
9,Union Creek,42.910278,-122.450000,,,Vault toilet,Vault toilet


In [36]:
geojson_df = cg_data_clean[['title','latitude','longitude','description']]

In [37]:
geojson_df

,title,latitude,longitude,description
0,Mazama,42.865508,-122.166149,"Flush toilet,Shower"
1,Claude Lewis Sno-Park-USFS,43.063217,-122.342876,Vault toilet
2,Lost Creek,42.879145,-122.037881,Flush toilet
3,Hamaker,43.056641,-122.330322,Vault toilet
4,Huckleberry Mountain,42.877807,-122.337174,Vault toilet
5,Natural Bridge -USFS,42.889648,-122.463867,Vault toilet
6,Farewell Bend,42.919678,-122.433105,
7,Scott Creek,42.884588,-121.924772,Vault toilet
8,Thousand Springs Sno-Park-USFS,42.913045,-122.324644,Vault toilet
9,Union Creek,42.910278,-122.450000,Vault toilet


In [58]:
def geojson_from_row(x):
    feature = 
    geojson = "{ \"type\": \"Feature\", \"properties\": { \"title\":"
    geojson = geojson + "\"" + str(x[0]) + "\", \"description\":"
    geojson = geojson + "\"" + str(x[3]) + "\" } "
    geojson = geojson + "\"geometry\": { \"type\": \"Point\", \"coordinates\":"
    geojson = geojson + "[" + str(x[2]) + "," + str(x[1]) + "] } }"
    return geojson

In [136]:
collection = {'type':'FeatureCollection', 'features':[]}

In [137]:
# expects
# title = x[0]
# latitude = x[1]
# longitude = x[2]
# description = x[3]
def feature_from_row(x):
    title = x[0]
    latitude = x[1]
    longitude = x[2]
    description = x[3]
    feature = { 'type': 'Feature', 
               'properties': { 'title': '', 'description': '', 'marker-size':'small'},
               'geometry': { 'type': 'Point', 'coordinates': []}
               }
    feature['geometry']['coordinates'] = [longitude, latitude]
    feature['properties']['title'] = title
    feature['properties']['description'] = description
    collection['features'].append(feature)
    return feature

In [138]:
geojson_series = geojson_df.apply(lambda x: feature_from_row(x), axis=1)

In [131]:
collection

{'features': [{'geometry': {'coordinates': [[-122.166149, 42.865508]],
    'type': 'Point'},
   'properties': {'description': 'Flush toilet,Shower',
    'marker-size': 'small',
    'title': 'Mazama'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-122.342876, 43.063217]], 'type': 'Point'},
   'properties': {'description': 'Vault toilet',
    'marker-size': 'small',
    'title': 'Claude Lewis Sno-Park-USFS'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-122.037881, 42.879145]], 'type': 'Point'},
   'properties': {'description': 'Flush toilet',
    'marker-size': 'small',
    'title': 'Lost Creek'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-122.330322, 43.056641]], 'type': 'Point'},
   'properties': {'description': 'Vault toilet',
    'marker-size': 'small',
    'title': 'Hamaker'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[-122.337174, 42.877807]], 'type': 'Point'},
   'properties': {'description': 'Vault toilet',
    'marker-size': 'sm

In [121]:
from pandas import json

In [132]:
with open('collection.geojson', 'w') as outfile:
    json.dump(collection, outfile)

In [133]:
test = pd.read_json('https://raw.githubusercontent.com/gizm00/blog_code/master/appendto/python_maps/collection.geojson')

In [134]:
test

,features,type
0,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
1,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
2,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
3,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
4,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
5,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
6,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
7,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
8,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
9,"{'type': 'Feature', 'geometry': {'type': 'Poin...",FeatureCollection
